### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/data/20220507_P183_octopus_SYTOXG/Atlas/P183_536BP_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/data/20220507_P183_octopus_SYTOXG/Atlas/mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "P183_536BP_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

# aff = np.array([[ 1.0, 0, 0],
#                 [ 0, 1.0, 0],
#                 [ 0, 0, 1.0]], dtype='float')
aff = np.array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.76604444, -0.64278761],
       [ 0.        ,  0.64278761,  0.76604444]], dtype="float")
transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/220620_P202_skeletonization/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,4",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=12)
R.set_fixed_image(os.path.join(tmpdir, "P183_536BP_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/220620_P202_skeletonization/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/220620_P202_skeletonization/ants/F2M_,/flash/ReiterU/tmp/220620_P202_skeletonization/ants/Warped.nii.gz,/flash/ReiterU/tmp/220620_P202_skeletonization/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/220620_P202_skeletonization/ants/P183_536BP_10um_masked.nii,/flash/ReiterU/tmp/220620_P202_skeletonization/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/220620_P202_skeletonization/ants/P183_536BP_10um_masked.nii,/flash/ReiterU/tmp/

DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 2DIAGNOSTIC,     1, -3.057583231655e-02, 1.797693134862e+308, 3.1256e+00, 3.1256e+00,
 2DIAGNOSTIC,     2, -3.059532377218e-02, 1.797693134862e+308, 3.1713e+00, 4.5646e-02,
 2DIAGNOSTIC,     3, -3.063508834388e-02, 1.797693134862e+308, 3.2213e+00, 5.0022e-02,
 2DIAGNOSTIC,     4, -3.070794516615e-02, 1.797693134862e+308, 3.2800e+00, 5.8670e-02,
 2DIAGNOSTIC,     5, -3.079199381107e-02, 1.797693134862e+308, 3.3359e+00, 5.5905e-02,
 2DIAGNOSTIC,     6, -3.082462063890e-02, 1.797693134862e+308, 3.3928e+00, 5.6945e-02,
 2DIAGNOSTIC,     7, -3.083370475743e-02, 1.797693134862e+308, 3.4451e+00, 5.2290e-02,
 2DIAGNOSTIC,     8, -3.083843018078e-02, 1.797693134862e+308, 3.5017e+00, 5.6562e-02,
 2DIAGNOSTIC,     9, -3.084321574041e-02, 1.797693134862e+308, 3.5716e+00, 6.9945e-02,
 2DIAGNOSTIC,    10, -3.086457433109e-02, 1.797693134862e+308, 3.6628e+00, 9.1143e-02,
 2DIAGNOSTIC,    11, -3.086828146533e-02, 1.7976

 2DIAGNOSTIC,    60, -3.239991546153e-01, 4.536154506774e-05, 3.2037e+01, 3.6961e-01,
 2DIAGNOSTIC,    61, -3.241011341541e-01, 3.145899211931e-05, 3.2392e+01, 3.5525e-01,
 2DIAGNOSTIC,    62, -3.241995751814e-01, 2.092229475470e-05, 3.2756e+01, 3.6394e-01,
 2DIAGNOSTIC,    63, -3.249767373289e-01, 3.104368674133e-05, 3.3111e+01, 3.5472e-01,
 2DIAGNOSTIC,    64, -3.252016720464e-01, 4.291541116042e-05, 3.3512e+01, 4.0104e-01,
 2DIAGNOSTIC,    65, -3.261579270726e-01, 6.423216867102e-05, 3.3870e+01, 3.5831e-01,
 2DIAGNOSTIC,    66, -3.268862417094e-01, 9.036831764082e-05, 3.4339e+01, 4.6865e-01,
 2DIAGNOSTIC,    67, -3.270893180569e-01, 1.137787616578e-04, 3.4905e+01, 5.6619e-01,
 2DIAGNOSTIC,    68, -3.271073904194e-01, 1.319961821066e-04, 3.5308e+01, 4.0292e-01,
 2DIAGNOSTIC,    69, -3.274160703510e-01, 1.484107814642e-04, 3.5696e+01, 3.8831e-01,
 2DIAGNOSTIC,    70, -3.279797728471e-01, 1.649025032303e-04, 3.6156e+01, 4.5988e-01,
 2DIAGNOSTIC,    71, -3.291442316371e-01, 1.8620390789

 2DIAGNOSTIC,    23, -4.050385621033e-01, 2.607700952209e-04, 1.1530e+02, 2.0576e+00,
 2DIAGNOSTIC,    24, -4.050382754993e-01, 1.953858663180e-04, 1.1743e+02, 2.1287e+00,
 2DIAGNOSTIC,    25, -4.050380615613e-01, 1.288959469796e-04, 1.1955e+02, 2.1206e+00,
 2DIAGNOSTIC,    26, -4.050380183178e-01, 6.308669911851e-05, 1.2167e+02, 2.1224e+00,
  Elapsed time (stage 0): 1.2373e+02


Stage 1
  iterations = 500x500x100x30
  convergence threshold = 1.0000e-06
  convergence window size = 10
  number of levels = 4
  using the CC metric (radius = 4, weight = 1.0000e+00, use gradient filter = 0)
  Shrink factors (level 1 out of 4): [8, 8, 8]
  Shrink factors (level 2 out of 4): [4, 4, 4]
  Shrink factors (level 3 out of 4): [2, 2, 2]
  Shrink factors (level 4 out of 4): [1, 1, 1]
  smoothing sigmas per level: [3, 2, 1, 0]
  Using default NONE metricSamplingStrategy

*** Running SyN registration (varianceForUpdateField = 4.0000e+00, varianceForTotalField = 2.0000e-01) ***

XXDIAGNOSTIC,Iteration,

 1DIAGNOSTIC,    84, -8.655279781865e-01, 1.580019743564e-05, 9.2510e+01, 9.4074e-01,
 1DIAGNOSTIC,    85, -8.656526830052e-01, 1.469368970483e-05, 9.3503e+01, 9.9236e-01,
 1DIAGNOSTIC,    86, -8.657312729614e-01, 1.344984136597e-05, 9.4542e+01, 1.0389e+00,
 1DIAGNOSTIC,    87, -8.658054854236e-01, 1.215394702990e-05, 9.5601e+01, 1.0593e+00,
 1DIAGNOSTIC,    88, -8.658424088345e-01, 1.064735745114e-05, 9.6659e+01, 1.0583e+00,
 1DIAGNOSTIC,    89, -8.658709319722e-01, 9.035790665825e-06, 9.7689e+01, 1.0302e+00,
 1DIAGNOSTIC,    90, -8.658808277323e-01, 7.355865781124e-06, 9.8609e+01, 9.1976e-01,
 1DIAGNOSTIC,    91, -8.658753577958e-01, 5.688626105303e-06, 9.9525e+01, 9.1571e-01,
 1DIAGNOSTIC,    92, -8.658854056734e-01, 4.165681454829e-06, 1.0044e+02, 9.1965e-01,
 1DIAGNOSTIC,    93, -8.658765426060e-01, 2.871620921815e-06, 1.0141e+02, 9.6926e-01,
 1DIAGNOSTIC,    94, -8.658291092931e-01, 1.702222877797e-06, 1.0254e+02, 1.1273e+00,
 1DIAGNOSTIC,    95, -8.657678693590e-01, 6.5437238743

 1DIAGNOSTIC,    84, -8.958955408548e-01, 4.126324535487e-06, 7.6878e+02, 8.2410e+00,
 1DIAGNOSTIC,    85, -8.959020415691e-01, 3.762769047280e-06, 7.7698e+02, 8.2024e+00,
 1DIAGNOSTIC,    86, -8.959774674588e-01, 3.711400856398e-06, 7.8541e+02, 8.4312e+00,
 1DIAGNOSTIC,    87, -8.959793428851e-01, 3.413010597803e-06, 7.9365e+02, 8.2341e+00,
 1DIAGNOSTIC,    88, -8.960543478354e-01, 3.432795174787e-06, 8.0139e+02, 7.7388e+00,
 1DIAGNOSTIC,    89, -8.961221008804e-01, 3.437442419471e-06, 8.0921e+02, 7.8241e+00,
 1DIAGNOSTIC,    90, -8.961187148321e-01, 3.421760456254e-06, 8.1805e+02, 8.8447e+00,
 1DIAGNOSTIC,    91, -8.961867796844e-01, 3.385715803212e-06, 8.2628e+02, 8.2217e+00,
 1DIAGNOSTIC,    92, -8.962482671635e-01, 3.535698883065e-06, 8.3412e+02, 7.8402e+00,
 1DIAGNOSTIC,    93, -8.963066723377e-01, 3.565258896528e-06, 8.4168e+02, 7.5655e+00,
 1DIAGNOSTIC,    94, -8.963595643369e-01, 3.725617316401e-06, 8.5059e+02, 8.9097e+00,
 1DIAGNOSTIC,    95, -8.964089991495e-01, 3.7392694309

 1DIAGNOSTIC,    61, -8.952450442313e-01, 6.971467974046e-06, 4.5442e+03, 6.1503e+01,
 1DIAGNOSTIC,    62, -8.953017757071e-01, 6.528062572318e-06, 4.6001e+03, 5.5974e+01,
 1DIAGNOSTIC,    63, -8.953590052228e-01, 6.368550624542e-06, 4.6615e+03, 6.1368e+01,
 1DIAGNOSTIC,    64, -8.954140984926e-01, 6.223411990629e-06, 4.7194e+03, 5.7848e+01,
 1DIAGNOSTIC,    65, -8.954714327703e-01, 6.103826087792e-06, 4.7752e+03, 5.5829e+01,
 1DIAGNOSTIC,    66, -8.955265734488e-01, 5.996437244308e-06, 4.8313e+03, 5.6142e+01,
 1DIAGNOSTIC,    67, -8.955804793170e-01, 5.888079434822e-06, 4.8895e+03, 5.8122e+01,
 1DIAGNOSTIC,    68, -8.956327945770e-01, 5.766305873331e-06, 4.9465e+03, 5.6999e+01,
 1DIAGNOSTIC,    69, -8.956841112843e-01, 5.626868338694e-06, 5.0045e+03, 5.8015e+01,
 1DIAGNOSTIC,    70, -8.957369852660e-01, 5.488407191832e-06, 5.0644e+03, 5.9964e+01,
 1DIAGNOSTIC,    71, -8.957868749771e-01, 5.339635053978e-06, 5.1225e+03, 5.8052e+01,
 1DIAGNOSTIC,    72, -8.958354817317e-01, 5.1839292880

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/220620_P202_skeletonization/ants', 'deigo:/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/analysis'], returncode=0)